In [2]:
import os
import time
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
pd.set_option('display.width', 400)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import*
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
import datetime as dt

In [5]:
data = pd.read_excel('进单历史数据_v20221122.xlsx',header=1).fillna(0)
data = data.iloc[:-1,1:-3]

In [6]:
data.head()

,物料代码_加密,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210
0,9DAA97026B7940FC56EB1C441E8B7E1F,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,86.71000,26.9698,45.2686,321.006401,339.783599,1308.842598,1275.85393,359.727498,475.146285,921.105366,913.918929
1,AA2BBBC4CE6F84538F43FE3F113A8161,0.0,0.0,0.0,503.62065,885.721123,1060.232471,57.330261,99.3667,216.7152,361.8498,410.826,367.95912,106.9822,170.7290,625.426224,348.627873,271.230824,536.52560,426.015201,358.826915,599.193026,327.379586
2,05A9182A08E8EE84EC4CB499FE955AF6,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.00000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,113.153465,145.866007,163.833616
3,4F98845EABDC7CD3AE528FAEB0343587,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.00000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,612.681300,304.676401,196.291208
4,969FBAD82B65F8AEBD532D5FF524F752,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.00000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,117.812640,114.829803,57.536812


In [7]:
df_agg = data.groupby('物料代码_加密').agg('sum')

In [8]:
df_agg

,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210
物料代码_加密,,,,,,,,,,,,,,,,,,,,,,
00053532195C060323E6DC869F434F4C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00780,0.000000,0.000000,0.000000,0.000000
00082D3A5BAD08670FC8C41BE6A11C16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67.506062,21.044360,11.885640,11.16960,32.589100,18.654750,9.311680,12.383120
000E155E93AC7F454684F7B48B59C169,0.000000,0.000000,0.048561,0.024280,0.024280,0.024280,0.072841,0.218523,0.072840,0.242804,0.218522,0.024280,0.000000,0.000000,0.048560,0.121401,0.169962,0.02428,0.146167,0.121402,0.167535,0.108048
000E269A3BCF58BA0CC3422E98108DA4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007022,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
000F2ED450F85D6999BC5FCC7DC68462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FFDDA807A21B0ADFF95470E3557275FC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.073500,0.000000,0.000000
FFEA8E4C89F9DB5FC826E6696773BB9E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.450576,6.960406,0.000000,0.000000,7.162128,54.918144,32.420736,15.63264,22.565376,26.613248,27.628346,78.163201
FFEEFD85201D071DE97308FBF109A482,0.000000,0.000000,0.000000,0.028313,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [9]:
sku = np.array(['sku'+str(i+1) for i in range(len(df_agg))])
wuliao_raw = df_agg.index
new_df = []

In [12]:
for t in df_agg.columns:
    jindan = df_agg[t].values
    month = np.array([str(t) for i in range(len(df_agg))])
    new_data = np.stack((month,sku,jindan))
    new_df.append(new_data)

In [13]:
new_df

[array([['202101', '202101', '202101', ..., '202101', '202101', '202101'],
        ['sku1', 'sku2', 'sku3', ..., 'sku15999', 'sku16000', 'sku16001'],
        ['0.0', '0.0', '0.0', ..., '0.0', '0.008484', '0.0']], dtype='<U32'),
 array([['202102', '202102', '202102', ..., '202102', '202102', '202102'],
        ['sku1', 'sku2', 'sku3', ..., 'sku15999', 'sku16000', 'sku16001'],
        ['0.0', '0.0', '0.0', ..., '0.0', '0.002828', '0.0']], dtype='<U32'),
 array([['202103', '202103', '202103', ..., '202103', '202103', '202103'],
        ['sku1', 'sku2', 'sku3', ..., 'sku15999', 'sku16000', 'sku16001'],
        ['0.0', '0.0', '0.048561', ..., '0.0', '0.005713', '0.0']],
       dtype='<U32'),
 array([['202104', '202104', '202104', ..., '202104', '202104', '202104'],
        ['sku1', 'sku2', 'sku3', ..., 'sku15999', 'sku16000', 'sku16001'],
        ['0.0', '0.0', '0.02428', ..., '0.028313', '0.0', '0.0']],
       dtype='<U32'),
 array([['202105', '202105', '202105', ..., '202105', '202105', '

In [14]:
new_df = np.hstack(new_df)
new_df = new_df.T

In [15]:
new_df

array([['202101', 'sku1', '0.0'],
       ['202101', 'sku2', '0.0'],
       ['202101', 'sku3', '0.0'],
       ...,
       ['202210', 'sku15999', '0.0'],
       ['202210', 'sku16000', '0.0032516'],
       ['202210', 'sku16001', '0.0']], dtype='<U32')

In [32]:
df_test = df_agg.stack().reset_index()

In [18]:
import subprocess

In [27]:
subprocess.run('python -m http.server 8001',shell=True).decode('utf-8').split('\n')

Serving HTTP on :: port 8001 (http://[::]:8001/) ...

Keyboard interrupt received, exiting.


KeyboardInterrupt: 

In [28]:
res = subprocess.Popen("python -m http.server 8001",shell=True,stdout=subprocess.PIPE,stderr=subprocess.PIPE)

In [29]:
res.stdout

<_io.BufferedReader name=78>

In [30]:
res.terminate()

In [31]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [7]:
from statsforecast.utils import AirPassengersDF
from tqdm.autonotebook import tqdm
import pandas as pd

In [3]:
Y_df = AirPassengersDF

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/jnagura/Energy-consumption-prediction-analysis/master/PJM_Load_hourly.csv')

URLError: <urlopen error [Errno 61] Connection refused>

In [1]:
import pandas as pd
df = pd.DataFrame({'A': [1, 2, None, 4],
                   'B': [None, None, None, None]})

In [2]:
df.columns[df.isna().all()]

Index(['B'], dtype='object')

In [3]:
import pandas as pd

In [5]:
df = pd.read_csv('./data/202101-202303.csv')

/Users/shnguo/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
len(df)

987053

In [13]:
len(df['item_code'].unique())

29646

In [8]:
29646*27

800442

In [10]:
nan_category_col = df.columns[df.isna().all()].tolist()

In [11]:
nan_category_col

['abc_code', 'abc_name', 'channel']

In [12]:
df.drop(columns=nan_category_col,inplace=True)

In [14]:
category_col=['office_name', 'cat_name', 'cat_code', 'big_class_id', 'pdt', 'second_pdt', 'pro_series', 'item_name', 'second_pdt_name', 'big_class_name', 'office', 'cat_id', 'pdt_name']

In [16]:
len(df[['item_code']+category_col].drop_duplicates())

29646

In [17]:
for c in category_col:
    print(c)
    print(len(df[['item_code']+[c]].drop_duplicates()))

office_name
29646
cat_name
29646
cat_code
29646
big_class_id
29646
pdt
29646
second_pdt
29646
pro_series
29646
item_name
29646
second_pdt_name
29646
big_class_name
29646
office
29646
cat_id
29646
pdt_name
29646


In [18]:
len(df[['item_code']].drop_duplicates())

29646